# Анализ данных и проверка методов обработки данных 2

In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from data_description import get_data_frame_settings
from processing import obj_to_cat


import DS_common.emissions as em
from DS_common.excel_writing import save_double_column_df
from sys import path

path.append(
    '/home/dranik/KFA/university/diplom_project/prog/data_processing/classification_power_predictor'
)

from classification_power_predictor\
import classification_power_predictor as cpp
from sklearn.metrics import roc_curve

# Загрузка данных

In [153]:
data = obj_to_cat(pd.read_csv(
    "/home/dranik/KFA/university/proc_1.csv"
))

# Очистка от выбросов

**Мы не удаляем строки - а прописываем выбросам пропуск**
Почему мы так делаем?

1) Что-бы не удалять больше чем нужно;<br>
2) Что-бы оставлось побольше данных - нет смысла много удалять пока не оценена классифицирующая способность.

Отрицительный стаж работы - бред!

In [154]:
data.loc[
    data['Работа последнее место стаж лет'] > 0,
    'Работа последнее место стаж лет'
] = np.NaN

Отрицительный доход - бред. Да и на 100 рублей не выжить.

In [155]:
data.loc[
    data['Работа уровень дохода BYR'] > 100,
    'Работа уровень дохода BYR'
] = np.NaN

Неположитеьная сумма договора - бред

In [156]:
data.loc[
    data['Сумма договора'] < 0,
    'Сумма договора'
] = np.NaN

Ежедневный платеж

In [157]:
data.loc[
    data['Ежедневный платеж'] < 0,
    'Ежедневный платеж'
] = np.NaN

**Показались подозлительными размахи переменных**

1.Количество фактов просрочки по основному долгу;<br>
2.Максимальное количество дней просрочки;<br>
3.Общее количество запросов в КБ;<br>
4.Сумма кредитных лимитов;<br>
5.Сумма договора;<br>
6.ежедневный платеж.

In [158]:
em_research_list = [
    "Количество фактов просрочки по основному долгу",
    "Максимальное количество дней просрочки",
    "Общее количество запросов в КБ",
    "Сумма кредитных лимитов", "Сумма договора",
    "Ежедневный платеж"
]

Рассмотрим описание данных

In [159]:
def get_describes(df):
    result = pd.DataFrame()
    for col in df:
        result[col] = df[col].describe()
    return result

get_describes(data[em_research_list]).\
to_excel("result/emissions_describes_initial.xlsx")

Ну гляза на максимальное значений и 75-е персентили все становиться понятно - проведем удаление по правилу из Википедии.

In [160]:
def cut_emissioins(col):
    return col[em.get_selcond_emiss_25_75(col, cut_type = 'right')]

data[em_research_list] = \
data[em_research_list].apply(cut_emissioins)

In [161]:
#get_describes(data[em_research_list])
get_describes(data[em_research_list]).\
to_excel("result/emissions_describes_new.xlsx")

# Классифицирующая способность

In [162]:
def cl_abby(data, file_name):
    object_data = data.copy()
    cat_cols = object_data.columns[object_data.dtypes == 'category']
    object_data[cat_cols] = object_data[cat_cols].astype("O")

    my_cpp = cpp(
        object_data.drop('Y', axis = 1), 
        object_data['Y'].replace({0:"Нет дефолта", 1:"Дефолт"}))
    my_cpp.update_predictors()
    
    writer = pd.ExcelWriter(file_name)
    save_double_column_df(my_cpp.result_DF,writer)
    writer.close()
    return my_cpp.result_DF

result_DF = cl_abby(data, "result/classify_abby/1.xlsx")

## По незначимости KS статистики выборку покидают

In [163]:
result_DF = pd.read_excel(
    "result/classify_abby/1.xlsx",
    index_col = 0, header=[0,1]
)


drop_index = result_DF[
    result_DF[("Дефолт","KS_p_val")] > 0.05
].index
drop_index.tolist()

['Количество детей',
 'Отношение к банку',
 'Работа последнее место стаж лет',
 'Работа уровень дохода BYR',
 'Количество действующих договоров обеспечения',
 'Число авто',
 'Есть авто']

In [164]:
data.drop(drop_index, inplace = True, axis = 1)

cl_abby(data, "result/classify_abby/2.xlsx")

### Показатели оказавшиеся хуже связанных

In [165]:
drop_cols = [
    'Ежедневный платеж',
    'Автомобиль год выпуска'
]

data.drop(drop_cols, inplace = True, axis = 1)

## Далее борьба с пропусками

Воинская служба много пропусков - но естесвенно понятно, что если мы укажем "невоеннообязанный" для всеж Ж у который пропуск то в 99% случаях окажемся правы.

In [166]:
data.loc[data['Пол'] == 'Ж', 'Воинская служба'] = "невоеннообязанный"

### Пропуски в числовых переменных

Найдем числовые переменные

In [167]:
numeric_cols = data.columns[
    (data.dtypes.apply(pd.api.types.is_numeric_dtype) & \
    np.invert(data.dtypes.apply(pd.api.types.is_bool_dtype)))
].drop(['Y', 'Дата рождения'])
numeric_cols

Index(['Автомобиль год выпуска1', 'Автомобиль год выпуска2',
       'Автомобиль год выпуска3', 'Количество иждивенцев',
       'Количество действующих кредитных договоров',
       'Количество запросов в КБ за последние 30 дней',
       'Количество фактов просрочки по основному долгу',
       'Максимальное количество дней просрочки',
       'Максимальный срок, на который заключался договор, в годах',
       'Общее количество запросов в КБ', 'Сумма кредитных лимитов',
       'Сумма договора', 'Срок кредита в днях'],
      dtype='object')

#### Пропуски в нули

Говорим что пропуски равны нулю

In [168]:
data[numeric_cols + '(пропуск => 0)'] = data[numeric_cols].apply(lambda x: x.fillna(0))

#### Перекодировка в бинарный

Если пропуск то false

In [169]:
data[numeric_cols + '(бинарный)'] \
= data[numeric_cols].apply(lambda x: x.isna())

#### Перекодировка на два класса по точке соотвесвтующей KS статистике и пропуски как отдельный уровень

In [170]:
def get_trio_col(col, y):
    
    # определение точки прелома
    norm_col = (col - np.min(col))/(np.max(col) - np.min(col))
    not_na_cond = np.invert(norm_col.isna())
    fpr, tpr, thresholds = roc_curve(y[not_na_cond], norm_col[not_na_cond])
    col_thres = col[thresholds[np.argmax(np.abs(fpr - tpr))] == norm_col].iloc[0]
    
    # распределение классов
    result = col.copy()
    result[not_na_cond & (col <= col_thres)] = "<=" + str(col_thres)
    result[not_na_cond & (col > col_thres)] = ">" + str(col_thres)
    
    return result.astype('category')

data[numeric_cols + '(разбивка по KS)'] = data[numeric_cols].apply(get_trio_col, y = data['Y'])

Сохраняем - только численные переменные и их свежие производные

In [171]:
numeric_and_derivative = numeric_cols.to_list() +\
(numeric_cols + '(пропуск => 0)').to_list() +\
(numeric_cols + '(бинарный)').to_list() +\
(numeric_cols + '(разбивка по KS)').to_list() + ['Y']

numeric_and_altern = cl_abby(
    data[
        numeric_and_derivative
    ],
    "result/classify_abby/3.xlsx"
)

numeric_and_altern.sort_index().to_excel("result/classify_abby/3.xlsx")

## Удаление лишних столбоцов

Постепенно из списка numeric_and_derivative будем выбрасывать столбцы что еще пригодяться

In [172]:
def save_col(col_name):
    numeric_and_derivative.remove(col_name)

Автомобили - хороший показатель, только очень много пропусков.

In [173]:
save_col('Автомобиль год выпуска1(разбивка по KS)')

Количесво дейсвующих кредитных договоров - лучше всего обнулять пропуски

In [174]:
save_col("Количество действующих кредитных договоров(пропуск => 0)")

И так далее по аналогии: количество иждивенцев не сохранить; сумма договора - по числам пропуск в ноль, но логически оказалось, что нулевая сумма договора - бред, там мало пропусков, потому оставим в изначальном виде.

In [175]:
save_col("Количество запросов в КБ за последние 30 дней(разбивка по KS)")
save_col("Количество фактов просрочки по основному долгу(пропуск => 0)")
save_col("Максимальное количество дней просрочки(пропуск => 0)")
save_col("Максимальный срок, на который заключался договор, в годах(разбивка по KS)")
save_col("Общее количество запросов в КБ(разбивка по KS)")
save_col("Срок кредита в днях(разбивка по KS)")
save_col("Сумма договора")
save_col("Сумма кредитных лимитов(пропуск => 0)")
save_col('Y')

In [177]:
data.drop(numeric_and_derivative, inplace = True, axis = 1)

Вернем производным столбцам названия их "родителей"

In [179]:

br_emptys = data.columns.str.find("(")
cond = (br_emptys != -1)

data.rename(columns = dict(zip(
    data.columns[cond],
    list(map(
        lambda x: x[0][:x[1]], zip(data.columns[cond], br_emptys[cond])
    ))
)), inplace = True)

## Удаляем то что осталось от пропусков

По тем, что остались численными надо удалить строки 

In [180]:
data.drop(data.loc[
    data[
        data.columns[data.dtypes.apply(pd.api.types.is_numeric_dtype)]
    ].apply(lambda x: any(x.isna()), axis = 1),:
].index, inplace = True)

Всем оставшимся делаем - нет данных

In [183]:
no_data_marker = "нет данных"
cat_cols = data.columns[data.dtypes == 'category']


data[cat_cols] = data[cat_cols].\
apply(lambda x: x.cat.add_categories(no_data_marker))

data.fillna(no_data_marker, inplace = True)

ValueError: new categories must not include old categories: {'нет данных'}

# Итак - финальные отчеты

In [184]:
cl_abby(data, "result/classify_abby/4.xlsx")

Нет дефолта            \
                                                           AUC        KS   
Воинская служба                                       0.650618  0.210338   
Недвижимость                                          0.529621  0.059242   
Образование                                           0.670562  0.259011   
Работа занимаемая должность                           0.652957  0.293796   
Семейное положение                                    0.661418  0.224163   
Собственная квартира                                  0.514276  0.028552   
Собственный дом                                       0.510551  0.021102   
Уголовная ответственность                             0.516347  0.032695   
Адрес проживания - Тип населенного пункта             0.633312  0.249214   
Гражданин РБ                                          0.603901  0.207261   
Дата рождения                                         0.566215  0.101329   
Был ли хоть один договор прекращен досрочно           0.618694  0.213041   
Наличие кредитной истории                             0.614339  0.228247   
Сумма договора                                        0.650847  0.275345   
Количество потребляемых банковских продуктов          0.595558  0.176293   
Пол                                                   0.542565  0.085131   
Социальная группа                                     0.628219   0.23705   
Код подразделения                                      0.70468  0.336869   
Количество действующих кредитных договоров            0.602079   0.19567   
Количество фактов просрочки по основному долгу        0.568068  0.126163   
Максимальное количество дней просрочки                0.567851  0.129881   
Сумма кредитных лимитов                               0.592405  0.183813   
Автомобиль год выпуска1                               0.516251  0.032406   
Количество запросов в КБ за последние 30 дней         0.625024  0.227041   
Максимальный срок, на который заключался догово...    0.614893  0.210735   
Общее количество запросов в КБ                        0.602359  0.192216   
Срок кредита в днях                                   0.661026  0.322051   

                                                                       \
                                                    KS_p_val rel_type   
Воинская служба                                          0.0        1   
Недвижимость                                             0.0        1   
Образование                                              0.0        1   
Работа занимаемая должность                              0.0        1   
Семейное положение                                       0.0        1   
Собственная квартира                                0.000033        1   
Собственный дом                                     0.004879        1   
Уголовная ответственность                           0.000001        1   
Адрес проживания - Тип населенного пункта                0.0        1   
Гражданин РБ                                             0.0        1   
Дата рождения                                            0.0       -1   
Был ли хоть один договор прекращен досрочно              0.0        1   
Наличие кредитной истории                                0.0        1   
Сумма договора                                           0.0       -1   
Количество потребляемых банковских продуктов             0.0        1   
Пол                                                      0.0        1   
Социальная группа                                        0.0        1   
Код подразделения                                        0.0        1   
Количество действующих кредитных договоров               0.0       -1   
Количество фактов просрочки по основному долгу           0.0       -1   
Максимальное количество дней просрочки                   0.0       -1   
Сумма кредитных лимитов                                  0.0       -1   
Автомобиль год выпуска1                             0.000001        1   
Количество запросов в

In [186]:
get_data_frame_settings(data)

,Тип данных,Область допустимых значений,Число допустимых значений,Число пропусков
Воинская служба,Номинативная,"военнослужащий, другое, не отслужил, невоенноо...",8,0
Недвижимость,Номинативная,"есть, нет, нет данных",3,0
Образование,Номинативная,"высшее, незаконченное высшее, неполное среднее...",6,0
Работа занимаемая должность,Номинативная,"государственный служащий, заместитель руководи...",9,0
Семейное положение,Номинативная,"вдовец/вдова, женат/замужем, повторный брак, р...",6,0
Собственная квартира,Номинативная,"есть, нет, нет данных",3,0
Собственный дом,Номинативная,"есть, нет, нет данных",3,0
Уголовная ответственность,Номинативная,"есть, нет, нет данных",3,0
Адрес проживания - Тип населенного пункта,Номинативная,"Агрогородок, Город, Городcкой поселок, Деревня...",11,0
Гражданин РБ,Номинативная,"Без гражданства, Другое, РБ, нет данных",4,0


In [187]:
data.to_csv("/home/dranik/KFA/university/proc_2.csv")